In [1]:
import torch
from qanta_util.qbdata import QantaDatabase
from lr_pytorch import GuessDataset, SimpleLogreg
from tfidf_guesser import TfidfGuesser, write_guess_json
from math import log

In [2]:
train = QantaDatabase('data/small.guesstrain.json')
dev = QantaDatabase('data/small.guessdev.json')

In [3]:
tfidf_guesser = TfidfGuesser()
tfidf_guesser.train(train, limit=-1)

In [4]:
fold = dev.guess_dev_questions

In [5]:
runs = fold[0].runs(200)
guesses = tfidf_guesser.guess(runs[0], max_n_guesses=5)
print(runs[0])
guesses

['Along with orbitons and holons, quasiparticles carrying this property are formed from electrons in Luttinger liquids, which carry density waves in charge and this property. Similar wave-like disturban', 'Along with orbitons and holons, quasiparticles carrying this property are formed from electrons in Luttinger liquids, which carry density waves in charge and this property. Similar wave-like disturbances in the collective structure of this property for a crystal lattice are quantized into magnons. This property separates orthohydrogen from parahydrogen, and its total value is 0 for singlet states ', 'Along with orbitons and holons, quasiparticles carrying this property are formed from electrons in Luttinger liquids, which carry density waves in charge and this property. Similar wave-like disturbances in the collective structure of this property for a crystal lattice are quantized into magnons. This property separates orthohydrogen from parahydrogen, and its total value is 0 for singl

[[('Mass', 0.18090644947344686),
  ('Refractive_index', 0.17805333129792436),
  ('Prime_number', 0.17756848677231601),
  ('Prime_number', 0.17756848677231601),
  ('Superconductivity', 0.16988461850422262)],
 [('Mass', 0.23940426124158237),
  ('Refractive_index', 0.23404879370343523),
  ('Prime_number', 0.23176757888704846),
  ('Prime_number', 0.23176757888704846),
  ('Superconductivity', 0.2223251790946655)],
 [('Refractive_index', 0.24990805338589098),
  ('Prime_number', 0.23817924348739059),
  ('Prime_number', 0.23817924348739059),
  ('Mass', 0.23542345600681913),
  ('Spin_(physics)', 0.2271269832172998)],
 [('Spin_(physics)', 0.2804451169252644),
  ('Spin_(physics)', 0.25181328547976783),
  ('Spin_(physics)', 0.24790080285401428),
  ('Spin_(physics)', 0.22886029369768995),
  ('Spin_(physics)', 0.2246240014010114)],
 [('Spin_(physics)', 0.318749252752448),
  ('Spin_(physics)', 0.2588195992166108),
  ('Spin_(physics)', 0.24740512039213625),
  ('Spin_(physics)', 0.2398361474149138),
  

In [6]:
for raw_guess, rr in zip(guesses, runs[0]):
    gg, ss = raw_guess[0]
    
    disambig = False
    if '(' in gg and ')' in gg:
        disambig = gg[gg.find("(")+1:gg.find(")")] in rr

    print(gg, ss, len(rr))
    # print(rr*ss)
    print([x[0] for x in raw_guess].count(gg))
    print(disambig)
    print()

Mass 0.18090644947344686 200
1
False

Mass 0.23940426124158237 400
1
False

Refractive_index 0.24990805338589098 600
1
False

Spin_(physics) 0.2804451169252644 800
5
False

Spin_(physics) 0.318749252752448 841
5
False



In [7]:
import requests

In [9]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"

params = {
    "action": "query",
    "format": "json",
    "titles": "Albert Einstein",
    "prop": "links",
    "pllimit": "max",
    "plnamespace": "0"
}

r = session.get(url=url, params=params)
data = r.json()
pages = data["query"]["pages"]

page_titles = []

for key, val in pages.items():
    for link in val["links"]:
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    for key, val in pages.items():
        for link in val["links"]:
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))
print(page_titles)

1142 titles found.
['1921 Nobel Peace Prize', '20th Century Press Archives', 'ADM formalism', 'A priori and a posteriori', 'Aage Bohr', 'Aarau', 'Abba Eban', 'Abdominal aortic aneurysm', 'Abdus Salam', 'Absent-minded professor', 'Absorption refrigerator', 'Action-angle variables', 'Adam Riess', 'Adiabatic invariant', 'Adolf Hitler', 'Aether (classical element)', 'African Americans', 'Afterlife', 'Agnosticism', 'Albert A. Michelson', 'Albert Calmette', "Albert Einstein's brain", 'Albert Einstein: Creator and Rebel', 'Albert Einstein (disambiguation)', 'Albert Einstein Archives', 'Albert Einstein Award', 'Albert Einstein House', 'Albert Einstein Medal', 'Albert Einstein Memorial', 'Albert Einstein Peace Prize', 'Albert Einstein World Award of Science', 'Albert Einstein in popular culture', 'Albert Fert', 'Albin Haller', 'Alchemy', 'Alexander Friedmann', 'Alexander Prokhorov', 'Alexei Abrikosov (physicist)', 'Alexis de Tocqueville', 'Alfonso XIII of Spain', 'Alfred Einstein', 'Alfred Kast